# Data Masters Case - Machine Learning Engineer: 
## Deploying Model in Cross-Workspace Scenario (Cross Cloud)
### Marcos Vinícius Lisboa Melo - BigData & Analytics - vinicius.lisboa@f1rst.com.br

At this notebok, we'll be able to load and deploy the model made at the other workspace. We can use the integrated MLFlow API features to deploy and evaluate the model performance. The TensorFlow model will be deployed by API, and be called by REST POST endpoint. It's important to be remember that TTensorFlow FX it's used by MLFlow Deploy to set the data format in the request. An instance of TFX is created by MLFlow in the deploy pipeline.

In [0]:
import mlflow
import cv2
import mlflow.pyfunc
import numpy as np
from mlflow.tracking.client import MlflowClient

## Verifiyng model uploaded in Azure
F1rst of all we can verify if the registred model by Azure Workspace, called `mnist_cnn_data_masters`, is correct seeing model informations, and if necessary, add a description and move forward the Model Status to: Stagging or Production as preferred. Bellow we'll load the model, using name and version.

In [0]:
model_name = 'mnist_cnn_data_masters'
model_version = 1
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

2022/12/11 20:01:11 WARNING mlflow.pyfunc: Encountered an unexpected error (AttributeError("module 'configparser' has no attribute '__version__'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


Creating local client:

In [0]:
client = MlflowClient()

So, we can correct load the model, now we can verify the stage and description:

In [0]:
model_version_details = client.get_model_version(
  name=model_name,
  version=model_version,
)
print("The current model stage is: '{stage}'".format(stage=model_version_details.current_stage))
print("The current model description is: '{description}'".format(description=model_version_details.description))

The current model stage is: 'None'
The current model description is: 'This model version was built using TensorFlow Keras. It is a convolutional neural network with three hidden layers.'


Adding a high-level description to model:

In [0]:
client.update_registered_model(
  name=model_name,
  description="Modelo de CNN desenvolvido para o case Data Masters Machine Learning Engineer."
)

The current model stage is: 'None'
The current model description is: ''


Add a model version description with information about the model architecture:

In [0]:
model_description = client.update_model_version(
  name=model_name,
  version=model_version,
  description="This model version was built using TensorFlow Keras. It is a convolutional neural network with three hidden layers."
)
print("The current model description is: '{description}'".format(description=model_description.description))

Out[23]: <ModelVersion: creation_timestamp=1670770695021, current_stage='None', description=('This model version was built using TensorFlow Keras. It is a convolutional '
 'neural network with three hidden layers.'), last_updated_timestamp=1670789814318, name='mnist_cnn_data_masters', run_id='', run_link='', source='dbfs:/databricks/mlflow/tmp-external-source/cae00547aafb4619a5869e59251542e0/model', status='READY', status_message='', tags={}, user_id='5535703946358135', version='1'>

Change model stage:

In [0]:
model_transiction = client.transition_model_version_stage(
  name=model_name,
  version=model_version,
  stage='Staging',
)
print("The current model stage is: '{stage}'".format(stage=model_transiction.current_stage))

The current model stage is: 'Staging'


## Creating a sample request to validate predict
After we verified all the model infos, we would like to perform a prediction in the model before deploy in a Cluster REST API. For that is necessary use a sample image test, at this case we need an image named `gray_image_test.jpg`.

In [0]:
from dbruntime.patches import cv2_imshow

expected_width = 28
expected_length = 28

image = cv2.imread('/dbfs/FileStore/shared_uploads/marcos.melo@cear.ufpb.br/gray_image_test.jpg')

width = expected_width/image.shape[0]
length = expected_length/image.shape[1]

image = cv2.resize(image0, (0, 0), fx = width, fy = length)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)

sample = [gray]
sample = np.array(sample)

At least we invoke the predict model passing the array that contains sample test image.

In [0]:
predictions = model.predict(sample)
classes = np.argmax(predictions, axis = 1)
print('Predict number is:' + str(classes))

1/1 [==============================] - 0s 29ms/step
Predict number is:[7]


**Now we can shift to "Model" window on Databricks and serve this model by REST API. If you preffer the deploy can be realized creating an API using a framework like FastAPI or using the MLServer by command line.**